## Load_Tokenization

In [8]:
import pandas as pd
import numpy as np
import re
import pickle

In [9]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
df = pd.read_csv('./data_l/ssodam2021_tokenized.csv',encoding='utf-8')

In [18]:
df.iloc[0]

num                                                                    1082901
board                                                                      익게2
title                                                                    해피뉴이어
text                                                      2021년은 모두 행복한 한해가 되길
writer                                                                      익명
upload_date                                                         2021/01/01
upload_time                                                              00:00
view                                                                    1343.0
likes                                                                    195.0
dislikes                                                                   0.0
comments                     앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...
comments_writer                                                            NaN
comments_cnt                                        

### Preprocessing

In [19]:
def get_noun_list_from_str(msg):
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [20]:
df = df.fillna('')
df.text_tokenized = df.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.title_tokenized = df.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.comments_tokenized = df.comments_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.comments_writer_tokenized = df.comments_writer_tokenized.apply(lambda x: get_noun_list_from_str(x))
df.upload_date = pd.to_datetime(df.upload_date)

In [82]:
df.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[]
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]"
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[]
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[]


In [3]:
import pickle
with open('./data_l/ssodam2021_tokenized_df.pkl','rb') as f:
    df = pickle.load(f)

## BoW

In [4]:
#불용어 load
with open('./data/stopwords.txt',encoding='utf-8') as f:
    stopwords = f.read().split()

In [370]:
#형태소 체크
limit = 0
for row in df.total_tokens:
    if limit > 10:
        break
    for tup in row:
        if tup[1]=='VerbPrefix':
            print(tup)
            limit += 1

('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('못', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('안', 'VerbPrefix')
('잘', 'VerbPrefix')
('안', 'VerbPrefix')


### 익게2 데이터로부터 BoW 생성

In [585]:
#Total Token's Bag of Words
corpus = []
for column in ['total_tokens']:
    for text in df.loc[df['board']=='익게2',column]:
        for word in text:
            if word[1]=='Noun':
                if word[0] not in stopwords:
                    corpus.append(word[0])

In [593]:
idx_total = {}
bow_total = []

for token in corpus:
    if token not in idx_total.keys():
        idx_total[token] = len(idx_total)
        bow_total.insert(len(idx_total)-1,1)
    else:
        bow_total[idx_total[token]] += 1
        
idx_key = list(idx_total.keys())

In [5]:
with open('./data/idx_total.pkl','rb') as f:
    idx_total = pickle.load(f)
with open('./data/bow_total.pkl','rb') as f:
    bow_total = pickle.load(f)
idx_key = list(idx_total.keys())

### 익게2의 각 날짜별 BoW 생성

In [135]:
from datetime import datetime, timedelta

#corpus for each day
cfd = pd.DataFrame(columns=['upload_date','BoW'])

for date in tqdm(pd.date_range(datetime(2021,1,1), datetime(2021,7,1))):
    bow = np.zeros(len(idx_total.keys()))
    
    for token in df.loc[(df.board=='익게2') & (df.upload_date==date),'total_tokens']:
        bow_temp = np.zeros(len(idx_total.keys()))
        for word in token:
            if (word[1]=='Noun') and (word[0] not in stopwords):
                try:
                    bow_temp[idx_total[word[0]]] += 1
                except:
                    print('Error occured')
        bow_temp[bow_temp>10] = 0
        bow += bow_temp            
    cfd.loc[len(cfd)] = [date, bow]

  0%|          | 0/182 [00:00<?, ?it/s]

In [6]:
with open('./data_l/cfd.pkl','rb') as f:
    cfd = pickle.load(f)

In [19]:
#BoW 매트릭스
BoW_mat = np.array([list(value) for value in cfd.BoW.values])
BoW_mat

array([[11.,  1., 29., ...,  0.,  0.,  0.],
       [ 0.,  0., 41., ...,  0.,  0.,  0.],
       [ 2.,  3., 42., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  3., 53., ...,  0.,  0.,  0.],
       [ 1.,  3., 54., ...,  1.,  2.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

###  Word별 일일 게시글 수 탐색 (시간 관계상 실패)

In [15]:
def return_words(x):
    output = []
    for token in x:
        if len(token)>1:
            output.append(token[0])
        else:
            pass
    return output

df['total_tokens'] = df.title_tokenized + df.text_tokenized + df.comments_tokenized + df.comments_writer_tokenized
df['_total_tokens'] = df['total_tokens'].apply(lambda x : return_words(x))

In [81]:
df.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized,total_tokens,_total_tokens
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[],"[(해피뉴이어, Noun), (2021년, Number), (은, Foreign),...","[해피뉴이어, 2021년, 은, 모두, 행복한, 한해, 가, 되길, 앗, ,, 성지..."
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[2021년, 새해, 복, 많이, 받으세요, 모든, 일이, 잘, 되, 기를, 12시..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Nou...","[첫, 글, 은, 내꼬, 예비, 회계사, 나, 다미, ㄹㅇ, 1등, 이네, ㅋㅋ, ..."
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[],"[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle), (1등, Nu...","[땡, ㅎㅎㅎㅎㅎㅎㅎ, 1등]"
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[],"[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[어디, 카운트, 다운, 하는, 곳, 없냐, 언, 제바, 껴, 232, 번, 불교,..."


__일별 토큰 합치기__

In [83]:
from datetime import datetime
#날짜 인덱스 정의
date_range = pd.date_range(datetime(2021,1,1), periods=181)
#빈 array 생성
day_total = [[] for i in range(181)]
for i, day in enumerate(date_range):
    # 해당 일자 탐색
    _df = df[df.upload_date==day].reset_index(drop=True)
    # 당일 토큰 병합
    for idx in range(len(_df)):
        # 익게2로 제한
        if _df.board[idx] == '익게2':
            row = _df.loc[idx]
            day_total[i] += row._total_tokens

__단어별 일일 게시글 수 계산__

In [46]:
def get_count(word ,X):
    count = 0
    for x in X:
        if x == word:
            count+=1
    return count

In [ ]:
import time

posts_total = np.zeros((181,len(idx_total)))

start_time = time.time()

from tqdm.notebook import tqdm

for j in tqdm(range(0,181)):
    _day_total = day_total[j]
    for i, word in tqdm(enumerate(idx_total)):
        counts = get_count(word, _day_total)
        posts_total[j,i] = counts
    if j%10 == 0:
        print(i,'번째 단어 처리중...',i-1000,'번째로부터',time.time()-start_time,'초 경과.')
        start_time = time.time()
        
#with open('./data/posts_total.pkl','wb') as f:
#    pickle.dump(posts_total, f)

In [56]:
df._total_tokens.apply(lambda x : get_count('해피뉴이어',x))

0        [해피뉴이어, 2021년, 은, 모두, 행복한, 한해, 가, 되길, 앗, ,, 성지...
1        [2021년, 새해, 복, 많이, 받으세요, 모든, 일이, 잘, 되, 기를, 12시...
2        [첫, 글, 은, 내꼬, 예비, 회계사, 나, 다미, ㄹㅇ, 1등, 이네, ㅋㅋ, ...
3                                         [땡, ㅎㅎㅎㅎㅎㅎㅎ, 1등]
4        [어디, 카운트, 다운, 하는, 곳, 없냐, 언, 제바, 껴, 232, 번, 불교,...
                               ...                        
77975    [아버지, 께서, 칼럼, 쓰셨는데, 댓글, 한, 번, 씩, 만, 달아줄, 수, 있을...
77976    [프로포폴, 하면, 기분, 이, 좋음, 하는거, 신기하네하면, 힙, 해보여서, 하는...
77977    [취업, 후기, 쓰려는데, 안, 올라간다, ㅠ, 손, 다쳐서, 폰, 으로, 열심히,...
77978    [간, 떨어지는, 동거, 보는, 사람, 아, 이번, 주, 기대했는데, 예고편, 에,...
77979    [오타루, 오, 마카, 세, 후기, 전, 에, 어떤, 다미, 가, 추천, 한, 거,...
Name: _total_tokens, Length: 77980, dtype: object

### 날짜별 인기 키워드 탐색

#### 지수 정의

급상승 지수
1. 전일 언급 횟수 a
2. 당일 언급된 횟수 b
3. 전체 기간 중 언급 횟수 c
4. 가중치 w0, w1, w2
5. 적당한 상수 K, 아주 작은 상수 E

w0 x log(1+(b - a)/(a + K1)) + w1 x tanh( 0.1 x (b - K2) ) - w2 x (1+E)^c

In [125]:
def relu(x):
    if x<0:
        return 0
    else:
        return x

# [ w0,w1, w2,  K1, K2, E]    
W = [1,0.5,-0.05, 5, 15, 0.0015]

def _hot_point(a,b,c):
    x0 = np.log(0.5*(b-a)/(a+W[3]) + 1)
    x1 = np.tanh(0.1*(b-W[4]))
    x2 = pow(1+W[5],c)
    
    return W[0]*x0, W[1]*x1, W[2]*x2
    
def hot_point(a,b,c):
    x,y,z = _hot_point(a,b,c)
    return x + y + z

def pp2(a,b,idx):
    c = BoW_mat[:,idx].sum()
    x, y, z = _hot_point(a,b,c)
    print(f'상승률지수 : {x}')
    print(f'언급지수   : {y}')
    print(f'패널티     : {z}')
    print(f'최종값     : {x+y+z}')

#### 날짜/단어별 지수 columns 생성

In [126]:
#각 날짜/단어별 급상승 지수 계산
cfd['points'] = [np.zeros(len(bow_total)) for i in range(len(cfd))]

from tqdm.notebook import tqdm

for i in tqdm(range(1,len(BoW_mat))):
    for j in range(len(bow_total)):
        cfd.points[i][j] = hot_point(BoW_mat[i-1,j],BoW_mat[i,j],BoW_mat[:,j].sum())

  0%|          | 0/181 [00:00<?, ?it/s]

In [127]:
#Points 매트릭스
Points = np.array([list(value) for value in cfd.points.values])
Points

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.33461232e-01, -5.94702049e-01, -2.20162633e+03, ...,
        -5.02649127e-01, -5.02724239e-01, -5.02649127e-01],
       [-3.08213663e-01, -2.09579585e-01, -2.20177705e+03, ...,
        -5.02649127e-01, -5.02724239e-01, -5.02649127e-01],
       ...,
       [-6.35880857e-01, -3.17793169e-01, -2.20189827e+03, ...,
        -5.02649127e-01, -5.02724239e-01, -5.02649127e-01],
       [-5.02349464e-01, -4.71943849e-01, -2.20177519e+03, ...,
        -3.97440644e-01, -2.98690135e-01, -3.97440644e-01],
       [-5.99259144e-01, -7.15330037e-01, -2.20334774e+03, ...,
        -5.89660504e-01, -6.56874919e-01, -5.89660504e-01]])

__일별 상위 10개/20개 키워드 추려내기__

In [128]:
#index 찾아 True로 저장할 ndarray 정의
top10s = np.zeros(Points.shape, dtype=bool)
top20s = np.zeros(Points.shape, dtype=bool)

for i in range(1,len(Points)):
    #날짜별 점수 상위 10개만 추림
    limit_10 = np.sort(Points[i,:])[-10]
    limit_20 = np.sort(Points[i,:])[-20]
    for j in range(Points.shape[1]):
        if Points[i,j]>=limit_20:
            #상위 20개에 대해 True값으로 저장
            top20s[i,j] = True
            
            if Points[i,j]>=limit_10:
                #상위 10개에 대해 True값으로 저장
                top10s[i,j] = True

top10s

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [129]:
#일별 상위 10개 키워드 저장
top10_keywords = [[[idx_key[int(j)],day] for j in np.arange(top10s.shape[1])[top10s[day]]] for day in range(len(top10s))]
#일별 상위 20개 키워드 저장
top20_keywords = [[[idx_key[int(j)],day] for j in np.arange(top20s.shape[1])[top20s[day]]] for day in range(len(top20s))]

__top10s의 BoW 생성__

In [130]:
top10s_idx = {}
top10s_dates = {}
top10s_bow = []

for day_keywords in top10_keywords:
    for keyword, day in day_keywords:
        if keyword not in top10s_idx.keys():
            top10s_idx[keyword]=len(top10s_idx)
            top10s_dates[keyword]=[day]
            top10s_bow.insert(len(top10s_idx)-1,1)
        else :
            top10s_dates[keyword].append(day)
            top10s_bow[top10s_idx[keyword]]+=1
            
top10s_bow = np.array(top10s_bow)

__BoW를 DataFrame화__

In [131]:
top10s_keys = [key for key, value in top10s_idx.items()]

df_10bow = pd.DataFrame(np.array([top10s_keys,top10s_bow]).T,
                      columns = ['keyword','count'])
df_10bow['count'] = df_10bow['count'].astype(int)
df_10bow['dates'] = top10s_dates.values()

df_10bow.head()

,keyword,count,dates
0,식당,1,[1]
1,방역,2,"[1, 90]"
2,서로,1,[1]
3,필수,3,"[1, 36, 49]"
4,조언,2,"[1, 157]"


In [136]:
df_10bow.sort_values('count',ascending=False).iloc[:30]

,keyword,count,dates
303,서울대,7,"[36, 53, 56, 65, 76, 95, 116]"
282,조교,6,"[33, 109, 115, 127, 131, 154]"
89,로욜라,6,"[10, 32, 82, 102, 116, 139]"
290,대출,6,"[34, 45, 48, 114, 140, 148]"
605,치킨,6,"[77, 96, 113, 134, 166, 172]"
39,수능,6,"[5, 24, 58, 149, 162, 169]"
643,확,6,"[83, 85, 87, 151, 173, 179]"
46,외모,6,"[5, 51, 69, 143, 161, 171]"
105,합격,5,"[12, 18, 19, 63, 98]"
79,한강,5,"[9, 119, 122, 125, 137]"


In [133]:
top20s_idx = {}
top20s_dates = {}
top20s_bow = []

for day_keywords in top20_keywords:
    for keyword, day in day_keywords:
        if keyword not in top20s_idx.keys():
            top20s_idx[keyword]=len(top20s_idx)
            top20s_dates[keyword]=[day]
            top20s_bow.insert(len(top20s_idx)-1,1)
        else :
            top20s_dates[keyword].append(day)
            top20s_bow[top20s_idx[keyword]]+=1
            
top20s_keys = [key for key, value in top20s_idx.items()]

df_20bow = pd.DataFrame(np.array([top20s_keys,top20s_bow]).T,
                      columns = ['keyword','count'])
df_20bow['count'] = df_20bow['count'].astype(int)
df_20bow['dates'] = top20s_dates.values()

df_20bow.head()

,keyword,count,dates
0,선택,1,[1]
1,진로,3,"[1, 76, 115]"
2,유지,2,"[1, 13]"
3,감사,2,"[1, 128]"
4,코,3,"[1, 76, 80]"


In [135]:
with open('./data/4th_bow_10.pkl','wb') as f:
    pickle.dump(df_10bow,f)
with open('./data/4th_bow_20.pkl','wb') as f:
    pickle.dump(df_20bow,f)

---

### 검증

In [124]:
arg = idx_total['조교']

for day in range(40,47):
    display([(idx_key[arg],Points[day,arg]) for arg in np.flip(np.argsort(Points[day])) if Points[day,arg]>0.3])
    print()

[('입결', 2.373325629631565),
 ('이과', 1.829739541065965),
 ('예수회', 1.3161102220241703),
 ('문과', 1.042272065285665),
 ('자랑', 0.9410646152597407),
 ('한의대', 0.7325395982206176),
 ('웹툰', 0.3951677320191124),
 ('설', 0.3664008148546232),
 ('진이뽀', 0.34768761330245007),
 ('기관', 0.33259918496428237),
 ('이벤트', 0.3066309636660743),
 ('균', 0.30105677041418116)]

[('북한', 0.5683121478189266), ('이사회', 0.4515376504475768)]

[('인스타', 1.3521135328386467),
 ('인구', 0.35738509357722503),
 ('행시', 0.32082851185080197)]

[('단계', 0.600608385919511), ('중국인', 0.40111022861203977)]

[('수강신청', 1.6269814306480028),
 ('수익률', 0.8048542208859417),
 ('짱깨', 0.6661299920438088),
 ('국민연금', 0.607261371999406),
 ('팩트', 0.4906162811328027)]

[('로그인', 1.0192774903094368),
 ('크롬', 0.893991607579015),
 ('창', 0.8820916760337818),
 ('시계', 0.7907923355860234),
 ('대기', 0.7045105346774294),
 ('수강신청', 0.703733313783917),
 ('익스', 0.6061168423167886),
 ('코딩', 0.598396131551214),
 ('장학금', 0.5878627684223631),
 ('용돈', 0.5538452391927635),
 ('네이버', 0.492956937234137),
 ('대출', 0.4829555521674384),
 ('거래', 0.4335939367729251),
 ('잠', 0.36184325936923345),
 ('조교', 0.348521545155721),
 ('번호', 0.33731729055047244),
 ('연습', 0.31993705781931364),
 ('생활비', 0.319006655310344),
 ('피방', 0.3013947361545994)]

[('저장', 0.6885981715723116),
 ('싸인', 0.4571032954869918),
 ('대여', 0.4142801478670041)]

In [24]:
Points[:,idx_total['메일']]

array([ 0.        ,  0.44733066, -1.38962605, -1.01933752, -1.49138052,
       -0.88678269, -1.38962605, -1.41769461, -1.4611197 , -1.5122089 ,
       -1.51999903, -1.22901625, -1.50277746, -1.41133781, -1.5122089 ,
       -1.32432643, -1.31801886, -1.2073045 , -1.44132309, -1.4611197 ,
       -1.51999903, -1.3204559 , -1.15364102, -1.47764   , -1.3077917 ,
       -1.47764   , -1.39212683, -1.5122089 , -1.15096477, -1.49138052,
       -1.40353203, -1.13831162, -1.5122089 , -0.98237653, -1.33246764,
       -1.41769461, -1.51999903, -1.41689872,  1.16670179, -1.38962605,
       -1.41769461, -1.41769461, -1.5122089 , -1.23647794, -1.22353859,
        0.8261009 ,  0.13473357,  0.7568307 , -1.47764   ,  0.07259906,
       -1.09302101, -1.50277746, -0.03635384, -1.41769461,  1.66388974,
        1.09311556,  0.44401627,  0.41977168, -1.38962605,  1.17658007,
        0.85539361,  0.43064285, -1.0180886 , -1.4611197 , -1.50277746,
       -0.98416094, -1.38962605, -1.38962605,  0.57112202, -1.46

In [111]:
df_10bow.sort_values('count',ascending=False)

,keyword,count,dates
333,수강신청,7,"[38, 44, 45, 46, 54, 56, 60]"
629,치킨,6,"[77, 96, 113, 134, 166, 172]"
91,로욜라,6,"[10, 32, 82, 102, 116, 139]"
291,조교,6,"[33, 109, 115, 127, 131, 154]"
672,확,5,"[83, 85, 87, 173, 179]"
...,...,...,...
560,교직원,1,[68]
561,출장,1,[68]
562,박병준,1,[68]
563,스터디,1,[69]


In [117]:
BoW_mat[:,idx_total['수강신청']][43:47]

array([  2.,  47., 115., 105.])

In [118]:
pp2(115,105,idx_total['수강신청'])

상승률지수 : -0.04255961441879589
언급지수   : 0.4999998874648379
패널티     : -0.2993702486581838
최종값     : 0.15807002438785822


In [63]:
def search_cd(word, x):
    try:
        return True if word in [y[0] for y in x] else False
    except:
        return False

def s_search(word, month, df2, show_urls=True):
    df_month = df2[df2.upload_date.apply(lambda x: True if x.month==month else False)]
    title_cd = df2.title_tokenized.apply(lambda x: search_cd(word,x))
    text_cd = df2.text_tokenized.apply(lambda x: search_cd(word,x))
    comments_cd = df2.comments_tokenized.apply(lambda x: search_cd(word,x))
    comments_writer_cd = df2.comments_writer_tokenized.apply(lambda x: search_cd(word,x))
    df = df_month[title_cd | text_cd | comments_cd | comments_writer_cd]
    if show_urls==True:
        [print('http://www.ssodam.com/content/'+str(number)) for number in df.num]
    return df

def numofposts(word, month):
    df = s_search(word,month,show_urls=False)
    return len(df)

In [64]:
s_search('떡밥',1,df)

http://www.ssodam.com/content/1084467
http://www.ssodam.com/content/1084741
http://www.ssodam.com/content/1085424
http://www.ssodam.com/content/1085435
http://www.ssodam.com/content/1086255
http://www.ssodam.com/content/1086941
http://www.ssodam.com/content/1088232
http://www.ssodam.com/content/1089908
http://www.ssodam.com/content/1089919
http://www.ssodam.com/content/1089925
http://www.ssodam.com/content/1089941
http://www.ssodam.com/content/1089953
http://www.ssodam.com/content/1090022
http://www.ssodam.com/content/1091137
http://www.ssodam.com/content/1092894
http://www.ssodam.com/content/1093030
http://www.ssodam.com/content/1093870
http://www.ssodam.com/content/1096319
http://www.ssodam.com/content/1096472
http://www.ssodam.com/content/1098601


,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
1067,1084467,익게2,해리포터는 떡밥을 알고 보니까,더 재밌어,익명,2021-01-03,13:58,333.0,1.0,0.0,"뭔데 나도 같이 알자, 해리포터랑 볼드모트랑 이복형제인거, 롸, 롸 익2 뭔솔, 그...",,5.0,"[(더, Noun), (재밌어, Adjective)]","[(해리포터, Noun), (는, Josa), (떡밥, Noun), (을, Josa...","[(뭔, Modifier), (데, Noun), (나도, Verb), (같이, Ad...",[]
1260,1084741,서르비,입법고시 법원행정고시 5급공채 변리사,자주 나오는 떡밥다미들의 선호도는,익명,2021-01-03,21:15,395.0,3.0,3.0,"솔직히 최근 경향으로는 가 제일 좋지, 내가 회계사라서 회계사 함ㅋ, 내가 사무관이...",,8.0,"[(자주, Noun), (나오는, Verb), (떡밥, Noun), (다미, Nou...","[(입법고시, Noun), (법원, Noun), (행정고시, Noun), (5, N...","[(솔직히, Adjective), (최근, Noun), (경향, Noun), (으로...",[]
1744,1085424,익게2,타대 언급글은 무조건 서르비임학점분포 기준,내 친구들 중에서 유독 학점 인증 많이 하는 몇 학교들을 보면서 그간 진심으로 궁금...,익명,2021-01-04,19:44,213.0,0.0,0.0,"그런글 몇개 있음 이미, 이미 여러번 식은 떡밥 , 인기글도 여럿갔었옹, 서연고 다...",아 몰랐었다 난 이제 알아서 꽤 충격맨날 궁금했었거든 내 친구들 다 맨날 에이만 받...,11.0,"[(내, Noun), (친구, Noun), (들, Suffix), (중, Noun)...","[(타대, Verb), (언급, Noun), (글, Noun), (은, Josa),...","[(그런, Modifier), (글, Noun), (몇개, Noun), (있음, A...","[(아, Exclamation), (몰랐었다, Verb), (난, Noun), (이..."
1755,1085435,익게2,잼일 너무 잼있어 ㅠㅠ,이건 ㄹㅈㄷ 영상이건 학점 떡밥에 맞는 영상서강대 교수님들 어디 없나요오오,익명,2021-01-04,19:54,50.0,2.0,0.0,", 항상알림 켜놨는데이분 정모하면 가고싶음 집도 성남이라","설마 진짜 교미, 언넝 대면 팬미팅 기원 ㅜㅜ",4.0,"[(이건, Noun), (ㄹㅈㄷ, KoreanParticle), (영상, Noun)...","[(잼일, Noun), (너무, Adverb), (잼있어, Verb), (ㅠㅠ, K...","[(,, Punctuation), (항상, Noun), (알림, Noun), (켜놨...","[(설마, Noun), (진짜, Noun), (교미, Noun), (,, Punct..."
2328,1086255,익게2,펜트하우스,이렇게 끝난다고,익명,2021-01-05,23:09,517.0,8.0,0.0,"시즌3 막판에 결론지을건가봐, 시즌1이 끝난거잖아, , 약간의 완결성이 있어야지ㅠㅠ...","차라리 차 전복하고 끝내지, 시즌제로 해서 그런가 갑자기 고무마 켁, 그렇긴한데 끝...",19.0,"[(이렇게, Adverb), (끝난다고, Verb)]","[(펜트하우스, Noun)]","[(시즌, Noun), (3, Number), (막판, Noun), (에, Josa...","[(차라리, Noun), (차, Noun), (전복, Noun), (하고, Josa..."
2808,1086941,익게2,법까지 건들기 시작하는 인공지능,변호사마저,익명,2021-01-07,00:11,1043.0,27.0,3.0,"ㄹㅇ 이러다가 유나바머 시즌2 찍는 건 아닌지 모르겠네, 진지충 유나바머는 아니지만...",데이터 처리같은건 가 하고 그걸 보면서 최소한의 인력의 회계사가 판단한다는 거지 회...,29.0,"[(변호사, Noun), (마저, Josa)]","[(법, Noun), (까지, Josa), (건들, Adverb), (기, Noun...","[(ㄹㅇ, KoreanParticle), (이러다가, Adverb), (유나바머, ...","[(데이터, Noun), (처리, Noun), (같은건, Adjective), (가..."
3696,1088232,익게2,갑자기 의대 약대 핫하네,왜고용시장 박살나서갈수록 직업들 하향평준화되서,익명,2021-01-09,12:11,606.0,1.0,0.0,"의대 약대는 늘 핫하지 않아, 그냥 정기 떡밥임, 이번에 약대가 수능으로 풀려서, ...",,6.0,"[(왜, Noun), (고용, Noun), (시장, Noun), (박살, Noun)...","[(갑자기, Noun), (의대, Noun), (약대, Noun), (핫, Noun...","[(의대, Noun), (약대, Noun), (는, Josa), (늘, Noun),...",[]
4821,1089908,익게2,근데 알페스랑 딥페이크,알페스랑 딥페이크 둘다 좆같은거 맞고둘다 당사자 입장에선 존나 끔찍하고 생길거같고...,익명,2021-01-12,11:31,605.0,26.0,30.0,"108990015이걸 보면 생각이 바뀜 합성 알페스도 있음ㅋㅋ그게 그거라고 생각함,...",ㅇㅇ ㅜ 합성 알페스는 그러면 이제 합성알페스가 아니고 딥페이크인거아님알페스 피해자...,26.0,"[(알, Noun), (페스, Noun), (랑, Josa), (딥, Noun), ...","[(근데, Adverb), (알, Noun), (페스, Noun), (랑, Josa...","[(108990015, Number), (이, Determiner), (걸, Nou...","[(ㅇㅇ, KoreanParticle), (ㅜ, KoreanParticle), (합..."
4829,1089919,익게2,번방은 알페스나 딥페랑은 아예 결이 다름,협박을 통해 실제 성착취행위가 이뤄진건데 ...,익명,2021-01-12,11:48,1711.0,68.0,32.0,"나도 동의애초에 왜 같이 거론되는지 몰겠음, ㅇㅈ 완전 같지는 않은듯김 변호사는 ...",ㄴㄴ 근데 자꾸 번방이랑 묶어서 비교하길래번방은 범죄의 심각성 정도를 떠나서 유형 ...,33.0,"[(협박, Noun), (을, Josa), (통해, Noun), (실제, Noun)...","[(번방, Noun), (은, Josa), (알, Noun), (페스나, Noun)...","[(나도, Verb), (동의, Noun), (애초, Noun), (에, Josa)...","[(ㄴㄴ, KoreanParticle), (근데, Adverb), (자꾸, Noun..."
4833,1089925,익게2,알페스 즐겼던 다미들도 눈팅중이겠네 ㅋㅋ,꾸준히 비추 박던거 너네 였냐 에라이 ㅋㅋ,익명,2021-01-12,12:01,3522.0,159.0,20.0,"다른 범죄와는 결이 다르다는 댓글 하나써주면 완벽, 당연시하던게 역겨울뿐 ㅋㅋㅋ, ...",,30.0,"[(꾸준히, Adjective), (비추, Verb), (박던거, Verb), (너...","[(알, Noun), (페스, Noun), (즐겼던, Verb), (다미, Noun...","[(다른, Noun), (범죄, Noun), (와는, Josa), (결, Noun)...",[]


## 월별 인기 키워드 찾기

### 작년 12월 데이터를 Load

In [80]:
tokens_20 = pd.read_csv('./data_l/ssodam2020_all_tokened_final.csv', encoding='utf-8')
#날짜 데이터를 datetime으로.
tokens_20.upload_date = pd.to_datetime(tokens_20.upload_date)
tokens_20.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comment_tokenized
0,813032,익게2,2020년 파이팅!!,모두들 파이팅,익명,2020-01-01,00:00,1375,168,0,"['올 첫글 ㅋㅋ', '아 너 뭔데 나보다 일찍 썼냐ㅡㅡ추천이나 먹어라', '첫글 ...",NaN,14,1,"[('파이팅', 'Noun')]","[('2020년', 'Number'), ('파이팅', 'Noun'), ('!!', ...","[(""['"", 'Punctuation'), ('올', 'Verb'), ('첫', '..."
1,813033,익게2,새해,ㅊㅊ,익명,2020-01-01,00:00,97,2,0,NaN,NaN,0,0,"[('ㅊㅊ', 'KoreanParticle')]","[('새해', 'Noun')]",NaN
2,813034,익게2,2020 첫 글 ㄱㅈㅇ,ㅇㅇ,익명,2020-01-01,00:00,128,1,0,NaN,NaN,0,0,"[('ㅇㅇ', 'KoreanParticle')]","[('2020', 'Number'), ('첫', 'Noun'), ('글', 'Nou...",NaN
3,813037,익게2,ㅅㅅ,ㅅㅅ,익명,2020-01-01,00:00,70,0,0,NaN,NaN,0,0,"[('ㅅㅅ', 'KoreanParticle')]","[('ㅅㅅ', 'KoreanParticle')]",NaN
4,813038,익게2,스물넷이다,뭐했다고 벌써,익명,2020-01-01,00:00,124,3,0,"['어? 너두?', '난방금전까지 스물넷..']",NaN,2,0,"[('뭐', 'Noun'), ('했다고', 'Verb'), ('벌써', 'Noun')]","[('스물넷', 'Noun'), ('이다', 'Josa')]","[(""['"", 'Punctuation'), ('?', 'Punctuation'), ..."


In [81]:
def get_noun_list_from_str(msg):
    try :
        return [tuple([re.sub("'",'',y) for y in re.findall('\'.*?\'',x)]) for x in re.findall('\(.*?\)',msg)]
    except :
        return np.NaN

In [82]:
#12월 데이터만 추려내어 처리
tokens_20_12 = tokens_20[tokens_20.upload_date.apply(lambda x: True if x.month==12 else False)]
tokens_20_12 = tokens_20_12.fillna('')
tokens_20_12.text_tokenized = tokens_20_12.text_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12.title_tokenized = tokens_20_12.title_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12.comment_tokenized = tokens_20_12.comment_tokenized.apply(lambda x: get_noun_list_from_str(x))
tokens_20_12['comments_writer_tokenized'] = [[] for _ in range(len(tokens_20_12))]
tokens_20_12.rename(columns={'comment_tokenized':'comments_tokenized'},inplace=True)
tokens_20_12.head()

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,popularity,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
161968,1058725,익게2,엽기붕어빵ㅋㅋ,붕어빵10개 1만 6천원ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...,익명,2020-12-01,00:06,124,0,0,['ㅈ같은 프리미엄화 좀 하지마라ㅠ'],,1,0,"[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)]","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[]
161969,1058727,익게2,다미들 푸우욱 잘 자라옹,졸리다냥,익명,2020-12-01,00:07,143,7,0,,,0,0,"[(졸리다냥, Verb)]","[(푸우, Noun), (욱, Noun), (자라, Noun), (옹, Noun)]",[],[]
161970,1058728,익게2,나 시 써봤는데 어때,거울 속에 한 사내가 서있다사내는 거울 속 자신의 모습이 볼품없다사내는 그런 자신의...,익명,2020-12-01,00:08,245,7,0,['거사사 너때 때멈 살그... 라고요???? 말이 너무 심하시네요!! 비추드립니다...,"['뭐라는거야ㅋㅋㅋㅋ', '저도 쓰고보니 이상 거울이랑 윤동주 자화상이랑 살짝 섞인...",8,0,"[(거울, Noun), (속, Noun), (사내, Noun), (서있다, Verb...","[(시, Noun), (써, Verb), (봤는데, Verb)]","[("", ,), (사사, Noun), (때멈, Noun), (살그, Noun), (...",[]
161971,1058729,익게2,민법 한번 공부해 보고싶은데.. 인강 뭐 들어야할지 모르겠어ㅜ,세무사 민법이랑 법무사 민법...,익명,2020-12-01,00:09,221,0,2,"['로스쿨 민법 어때? - 로스쿨다미', '자기계발용으로는 적합한지 모르겠지만ㅋㅋㅋ...","['헤헤 그거 들어볼까?? 혹시 추천 강사님 물어봐도돼?ㅎ', 'ㅋㅋㅋㅋ']",8,0,"[(세무사, Noun), (민법, Noun), (법무사, Noun), (민법, No...","[(민법, Noun), (한번, Noun), (공부, Noun), (해, Verb)...","[("", ,), (로스쿨, Noun), (민법, Noun), (?, Punctuat...",[]
161972,1058730,익게2,가정내에서도 결국 사회생활하듯이 살아야할까,나는 엄마랑 감정적으로 친해지고 싶지 않아말하는 관점이 너어어어무 다르고 남 험담을...,익명,2020-12-01,00:10,376,11,0,"['솔직히 내가 그 상황이 아니니까 답은 알 수 없지만, 나라면 안 드려 이번에 드...",['진짜 고민이야 ㅠㅠ 정말 오늘 아침도 또 아빠욕하길래 못들어주겟다고 하고 나왔는...,11,0,"[(엄마, Noun), (랑, Josa), (감정, Noun), (적, Suffix...","[(가정, Noun), (내, Noun), (에서도, Josa), (사회생활, No...","[("", ,), (솔직히, Adjective), (내, Noun), (상황, Nou...",[]


In [83]:
tokens_20_12.columns

Index(['num', 'board', 'title', 'text', 'writer', 'upload_date', 'upload_time',
       'view', 'likes', 'dislikes', 'comments', 'comments_writer',
       'comments_cnt', 'popularity', 'text_tokenized', 'title_tokenized',
       'comments_tokenized', 'comments_writer_tokenized'],
      dtype='object')

In [84]:
df.columns

Index(['num', 'board', 'title', 'text', 'writer', 'upload_date', 'upload_time',
       'view', 'likes', 'dislikes', 'comments', 'comments_writer',
       'comments_cnt', 'text_tokenized', 'title_tokenized',
       'comments_tokenized', 'comments_writer_tokenized'],
      dtype='object')

In [85]:
df2 = pd.concat([tokens_20_12,df])

### 익게2 12월~6월간 BoW 생성

전체 데이터프레임을 월별로 groupby.

In [75]:
cfm_dict = dict.fromkeys(['upload_month','title_tokenized','text_tokenized','comments_tokenized','comments_writer_tokenized'])

for key in cfm_dict.keys():
    if key == 'upload_month':
        cfm_dict[key] = [12]+[i+1 for i in range(6)]
    else:
        cfm_dict[key] = [[] for i in range(7)]

cfm_dict

{'upload_month': [12, 1, 2, 3, 4, 5, 6],
 'title_tokenized': [[], [], [], [], [], [], []],
 'text_tokenized': [[], [], [], [], [], [], []],
 'comments_tokenized': [[], [], [], [], [], [], []],
 'comments_writer_tokenized': [[], [], [], [], [], [], []]}

In [658]:
_df = df2.loc[df2.board=='익게2']

for idx in range(len(_df)):
    if (idx+1)%10000==0:
        print('index :',idx,'processing...')
    row = _df.iloc[idx]
    #데이터프레임으로부터 월 인덱스 탐색
    month = 0 if row.upload_date.month==12 else row.upload_date.month
    #칼럼별 해당 인덱스에 해당되는 토큰 저장
    #50단어가 넘는 장문의 글은 제거하고 댓글과 제목만 남김.
    if len(row.text_tokenized) > 50:
        pass
    else :
        cfm_dict['text_tokenized'][month] += row.text_tokenized
    cfm_dict['title_tokenized'][month] += row.title_tokenized
    cfm_dict['comments_tokenized'][month] += row.comments_tokenized
    cfm_dict['comments_writer_tokenized'][month] += row.comments_writer_tokenized

index : 9999 processing...
index : 19999 processing...
index : 29999 processing...
index : 39999 processing...
index : 49999 processing...
index : 59999 processing...
index : 69999 processing...


In [659]:
cfm = pd.DataFrame(cfm_dict)
cfm['total_tokens'] = cfm.title_tokenized + cfm.text_tokenized + cfm.comments_tokenized + cfm.comments_writer_tokenized
cfm

,upload_month,title_tokenized,text_tokenized,comments_tokenized,comments_writer_tokenized,total_tokens
0,12,"[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)...","[(붕어빵, Noun), (10, Number), (개, Noun), (1만, Nu...","[("", ,), (ㅈ, KoreanParticle), (같은, Adjective),...",[],"[(엽기, Noun), (붕어빵, Noun), (ㅋㅋ, KoreanParticle)..."
1,1,"[(해피뉴이어, Noun), (2021년, Number), (새해, Noun), (...","[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)...","[(해피뉴이어, Noun), (2021년, Number), (새해, Noun), (..."
2,2,"[(않이, Verb), (모해, Noun), (따고, Verb), (벌써, Noun...","[(2월, Number), (이냐, Foreign), (지인, Noun), (이, ...","[(난, Noun), (마카롱, Noun), (세트, Noun), (했아, Verb...","[(오, Noun), (좋다, Adjective), (고마워, Adjective),...","[(않이, Verb), (모해, Noun), (따고, Verb), (벌써, Noun..."
3,3,"[(약, Noun), (19, Number), (저가, Noun), (코스프레, N...","[(취미, Noun), (로, Josa), (배워, Verb), (보고싶어, Ver...","[(진짜, Noun), (이형, Noun), (꾸준하네, Adjective), (,...","[(고마워, Adjective), (덕분, Noun), (에, Josa), (등업,...","[(약, Noun), (19, Number), (저가, Noun), (코스프레, N..."
4,4,"[(만우절, Noun), (이, Josa), (벌써, Noun), (끝나, Verb...","[(아쉽다, Adjective), (아쉬워, Adjective), (연애, Noun...","[(스타트, Noun), (가, Josa), (좋구만, Adjective), (ㄱㄱ...","[(나, Noun), (에게, Josa), (있다고, Adjective), (,, ...","[(만우절, Noun), (이, Josa), (벌써, Noun), (끝나, Verb..."
5,5,"[(2021년, Number), (5월, Number), (첫, Noun), (글,...","[(새로운, Adjective), (달도, Noun), (희망, Noun), (차게...","[(뭔, Modifier), (데, Noun), (벌써, Noun), (5월, Nu...","[(아, Exclamation), (진짜, Noun), (로, Josa), (그럼,...","[(2021년, Number), (5월, Number), (첫, Noun), (글,..."
6,6,"[(지금, Noun), (퇴근, Noun), (하는, Verb), (길, Noun)...","[(입사, Noun), (1년, Number), (아직, Adverb), (안된, ...","[(화이팅, Noun), (시간, Noun), (이, Josa), (지나면, Ver...","[(전망, Noun), (좋고, Adjective), (과제, Noun), (하기,...","[(지금, Noun), (퇴근, Noun), (하는, Verb), (길, Noun)..."


In [660]:
#Total Token's Bag of Words
corpus_extended = []
for column in ['total_tokens']:
    for text in cfm[column]:
        for word in text:
            try:
                if word[1]=='Noun':
                    if word[0] not in stopwords:
                        corpus_extended.append(word[0])
            except:
                pass

In [661]:
idx_extended = {}
bow_extended = []

for token in corpus_extended:
    if token not in idx_extended.keys():
        idx_extended[token] = len(idx_extended)
        bow_extended.insert(len(idx_extended)-1,1)
    else:
        bow_extended[idx_extended[token]] += 1
        
idx_extended_key = list(idx_extended.keys())

In [14]:
'''
with open('./data/idx_extended.pkl','wb') as f:
    pickle.dump(idx_extended,f)
with open('./data/bow_extended.pkl','wb') as f:
    pickle.dump(bow_extended,f)
'''
with open('./data/idx_extended.pkl','rb') as f:
    idx_extended = pickle.load(f)
with open('./data/bow_extended.pkl','rb') as f:
    bow_extended = pickle.load(f)
idx_extended_key = list(idx_extended.keys())

###  Word별 게시글 수 탐색

In [983]:
def return_words(x):
    output = []
    for token in x:
        if len(token)>1:
            output.append(token[0])
        else:
            pass
    return output

df2['total_tokens'] = df2.title_tokenized + df2.text_tokenized + df2.comments_tokenized + df2.comments_writer_tokenized
df2['_total_tokens'] = df2['total_tokens'].apply(lambda x : return_words(x))

In [1554]:
from datetime import datetime
month_filter = []
for i in range(7):
    if i==0:
        month_filter.append((datetime(2020,12,1)<=df2.upload_date) & (df2.upload_date<datetime(2021,1,1)) & (df2.board=='익게2'))
    else:
        month_filter.append((datetime(2021,i,1)<=df2.upload_date) & (df2.upload_date<datetime(2021,i+1,1)) & (df2.board=='익게2'))

import time

posts_extended = np.zeros((7,len(idx_extended)))

start_time = time.time()

for i, word in enumerate(idx_extended):
    if i%1000 == 0:
        print(i,'번째 단어 처리중...',i-1000,'번째로부터',time.time()-start_time,'초 경과.')
        start_time = time.time()
    _all_posts = df2._total_tokens.apply(lambda x: True if word in x else False)
    for j in range(0,7):
        posts_extended[j,i] = _all_posts[month_filter[j]].sum()

In [8]:
with open('./data/posts_extended.pkl','rb') as f:
    posts_extended = pickle.load(f)

### 익게2 데이터로부터 월별 BoW 생성

__BoW 생성__

In [665]:
bows = []

for month in range(7):
    bow = np.zeros(len(idx_extended.keys()))
    
    for word in cfm.iloc[month].total_tokens:
        #()등 빈 단어들은 패스
        if len(word)!=2:
            continue
        if (word[1]=='Noun') and (word[0] not in stopwords):
            try:
                bow[idx_extended[word[0]]] += 1
            except:
                print('Error occured')
                
    bows.append(bow)

cfm['BoW'] = bows

In [17]:
'''
with open('./data_l/cfm.pkl','wb') as f:
    pickle.dump(cfm,f)
'''
with open('./data_l/cfm.pkl','rb') as f:
    cfm = pickle.load(f)

### 급상승 지수 계산

idx_total & bow_total로부터 각 월마다 키워드별 급상승지수 계산

월별 급상승 지수
1. 전월 언급 횟수 a
2. 당월 언급된 횟수 b
3. 전월 게시글 작성 횟수 c
3. 당월 게시글 작성 횟수 d
4. 전체 기간(2020.12~2021.6) 중 언급 횟수 e
5. 가중치 w0, w1, w2, w3, w4
6. 적당한 상수 K, 아주 작은 상수 E

w0 x sigmoid((b-a)/(a+K1)) + w1 x {tanh(0.05 x (b-K2)) - 0.5} + w2 x sigmoid((d-c)/(c+K3)) + w3 x {tanh(0.1 x (d-20)) - 0.7} + w4 x e^E

- 월별 등장 횟수는 일별 등장 횟수에 비해 양에 대한 기준치를 높여야 하므로, K1과 K2를 상향 조정

In [365]:
def relu(x):
    if x<0:
        return 0
    else:
        return x
    
def sigmoid2(x):
    return 1/( 1+np.exp( -(x-5) ) )

#   [w0,   w1,  w2,  w3,    w4,  K1,  K2, K3, E]    
W = [1.5, 0.3, 1.3, 0.5, -0.05, 10, 100, 10, 0.002]

def _hot_point3(a,b,c,d,e):
    x0 = sigmoid2((b-a)/(a+W[5]))
    x1 = np.tanh(0.05*(b-W[6]))-0.5
    x2 = sigmoid2((d-c)/(c+W[7]))
    x3 = np.tanh(0.1*(d-20))-0.7
    x4 = pow(1+W[8],e) - 1
    
    return W[0]*x0, W[1]*x1, W[2]*x2, W[3]*x3, W[4]*x4
    
def hot_point3(a,b,c,d,e):
    x,y,z,w,v = _hot_point3(a,b,c,d,e)
    return x + y + z + w + v

def pp3(a,b,c,d,idx):
    e = BoW_mat[:,idx].sum()
    x, y, z, w, v = _hot_point3(a,b,c,d,e)
    print(f'상승률지수 : {x}')
    print(f'언급지수   : {y}')
    print(f'게시글상승 : {z}')
    print(f'게시글수   : {w}')
    print(f'패널티     : {v}')
    print(f'최종값     : {x+y+z+w+v}')

In [366]:
#BoW 매트릭스
BoW_mat = np.array([list(value) for value in cfm.BoW.values])
BoW_mat

array([[ 36., 194.,   6., ...,   0.,   0.,   0.],
       [ 18.,  29.,   4., ...,   0.,   0.,   0.],
       [  4.,  20.,   0., ...,   0.,   0.,   0.],
       ...,
       [  3.,  38.,   8., ...,   0.,   0.,   0.],
       [ 22.,  18.,   3., ...,   0.,   0.,   0.],
       [ 12.,  39.,   3., ...,   3.,   3.,   3.]])

In [367]:
#각 날짜/단어별 급상승 지수 계산
cfm['points'] = [np.zeros(len(bow_extended)) for i in range(len(cfm))]

for i in range(1,len(BoW_mat)):
    for j in range(len(bow_extended)):
        cfm.points[i][j] = hot_point3(BoW_mat[i-1,j],BoW_mat[i,j],
                                      posts_extended[i-1,j],posts_extended[i,j],
                                      BoW_mat[:,j].sum())

In [368]:
#Points 매트릭스
Points = np.array([list(value) for value in cfm.points.values])
Points

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.26481776, -1.2968079 , -1.26402542, ..., -1.26354717,
        -1.26354717, -1.26354717],
       [-1.27580816, -1.31301699, -1.26999508, ..., -1.26354717,
        -1.26354717, -1.26354717],
       ...,
       [-1.27342948, -1.28727906, -1.25261751, ..., -1.26354717,
        -1.26354717, -1.26354717],
       [-1.16918425, -1.29200404, -1.26651268, ..., -1.26354717,
        -1.26354717, -1.26354717],
       [-1.27101071, -1.26587024, -1.26285661, ..., -1.25525361,
        -1.25525361, -1.25525361]])

__추가 불용어 정의__

In [369]:
#도배된 단어 등 제외
for x_word in ['는','걍','을해','얀','인데','꿀꿀','끝내','쿠','원','쿠나','게로','사지','능','조만간','알폰소','아론']:
    Points[:,idx_extended[x_word]] = 0.

### 상위 키워드 추리기

In [370]:
#index 찾아 True로 저장할 ndarray 정의
top10s = np.zeros(Points.shape, dtype=bool)
top20s = np.zeros(Points.shape, dtype=bool)

for i in range(1,len(Points)):
    #날짜별 점수 상위 10개만 추림
    limit_10 = np.sort(Points[i,:])[-10]
    limit_20 = np.sort(Points[i,:])[-20]
    for j in range(Points.shape[1]):
        if Points[i,j]>=limit_20:
            #상위 20개에 대해 True값으로 저장
            top20s[i,j] = True
            
            if Points[i,j]>=limit_10:
                #상위 10개에 대해 True값으로 저장
                top10s[i,j] = True

top10s

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [371]:
#일별 상위 10개 키워드 저장
top10_keywords = [[idx_extended_key[int(j)] for j in np.arange(top10s.shape[1])[top10s[i]]] for i in range(len(top10s))]
#일별 상위 20개 키워드 저장
top20_keywords = [[idx_extended_key[int(j)] for j in np.arange(top20s.shape[1])[top20s[i]]] for i in range(len(top20s))]

In [372]:
top10_keywords

[[],
 ['눈사람', '능검', '공론', '재래시장', '페스', '페이크', '정인', '로토', '팬픽', '강민철'],
 ['증원', '쿼터', '뻔뻔', '연강', '괴롭힘', '뻔선', '뻔후', '폭로', '학폭', '학교폭력'],
 ['피셋', '결석', '벚꽃', '로나', '검찰', '토지', '신도시', '투기', '단태', '브레이브걸스'],
 ['파상', '벼락치기', '클로즈', '분포', '슈퍼리그', '비제', '권혁', '만우절', '서예지', '김정현'],
 ['참가자', '퇴소', '자진', '파이', '정민', '퍼즐', '이루리', '공혁준', '또즐', '니갸르'],
 ['부정행위', '텀페', '중회', '독일', '관회', '메리트', '광학', '팬덤', '유로', '소희']]

In [373]:
top20_keywords

[[],
 ['눈사람',
  '능검',
  '공론',
  '재래시장',
  '페스',
  '페이크',
  '손예진',
  '삽자루',
  '정인',
  '로토',
  '팬픽',
  '보따리',
  '강민철',
  '불평',
  '와이번스',
  '동포',
  '담배값',
  '냉면',
  '아나콘다',
  '오줌싸개'],
 ['증원',
  '물성',
  '성폭행',
  '쿼터',
  '설날',
  '뻔뻔',
  '사인',
  '연강',
  '괴롭힘',
  '뻔선',
  '연휴',
  '방관',
  '뻔후',
  '폭로',
  '학폭',
  '학교폭력',
  '탈세',
  '동물원',
  '김연경',
  '역군'],
 ['장관',
  '피셋',
  '에이프릴',
  '결석',
  '벚꽃',
  '로나',
  '검찰',
  '역주행',
  '토지',
  '신도시',
  '서진',
  '투기',
  '유신',
  '롤린',
  '단태',
  '브레이브걸스',
  '마사',
  '몰수',
  '하윤',
  '꼬북좌'],
 ['부정행위',
  '파상',
  '벼락치기',
  '테뱅',
  '클로즈',
  '토트넘',
  '분포',
  '슈퍼리그',
  '비제',
  '너비',
  '간담',
  '애정',
  '작',
  '윽박',
  '결식',
  '권혁',
  '만우절',
  '서예지',
  '진순',
  '김정현'],
 ['코난',
  '범인',
  '전기',
  '참가자',
  '학생회',
  '성당',
  '퇴소',
  '사물함',
  '실종',
  '자진',
  '파이',
  '어린이',
  '에어컨',
  '정민',
  '퍼즐',
  '이루리',
  '캣맘',
  '공혁준',
  '또즐',
  '니갸르'],
 ['부정행위',
  '경통',
  '텀페',
  '중회',
  '떡밥',
  '국무',
  '독일',
  '할미',
  '사프컴',
  '관회',
  '반타작',
  '메리트',
  '광학',
  '팬덤',
  '잉글랜드',
  '현주',
  '유로',

__월별 키워드 보기__

In [399]:
month = 1

for month in range(1,7):
    display([(idx_extended_key[arg],Points[month,arg]) for arg in np.flip(np.argsort(Points[month])) if Points[month,arg]>0.4])
    print()

[('능검', 2.260767814533165),
 ('눈사람', 1.6846429495194757),
 ('페스', 1.610230077040688),
 ('로토', 1.0773097870681805),
 ('팬픽', 1.0042855362774332),
 ('페이크', 1.0042390116242759),
 ('강민철', 0.8337913365931837),
 ('재래시장', 0.8249923193467364),
 ('정인', 0.7824369188747704),
 ('공론', 0.7237889856652208),
 ('냉면', 0.7000926140740011),
 ('오줌싸개', 0.6285326091369418),
 ('아나콘다', 0.5467134683125027),
 ('와이번스', 0.5331136670134958),
 ('보따리', 0.5063553456897618),
 ('삽자루', 0.4552971322576629)]

[('뻔뻔', 2.9060766694286704),
 ('학폭', 2.17729433603298),
 ('뻔선', 1.8340011697949712),
 ('학교폭력', 1.7158133778593483),
 ('쿼터', 1.5240046743778157),
 ('증원', 1.4487172599266391),
 ('뻔후', 1.238929017862722),
 ('폭로', 1.2205242316051437),
 ('괴롭힘', 1.1032633441401642),
 ('연강', 0.9081321681763606),
 ('동물원', 0.877751313243363),
 ('탈세', 0.8742128362653893),
 ('김연경', 0.8676058012943388),
 ('물성', 0.8389613842387756),
 ('역군', 0.8090530675135683),
 ('사인', 0.7851861517363047),
 ('연휴', 0.6764222193690317),
 ('방관', 0.6759785392670195),
 ('성폭행', 0.6582000882432335),
 ('설날', 0.6501837356569178),
 ('돌고래', 0.6475324203931211),
 ('매매혼', 0.645106973790332),
 ('강인원', 0.6242970899004467),
 ('박물관', 0.5779721602587583),
 ('뜻뜻', 0.5572805601042105),
 ('이진욱', 0.5127268180871906),
 ('읏', 0.4925825322501887),
 ('박혜수', 0.48565225162724185),
 ('명절', 0.48543285974844463),
 ('연가', 0.4368597579045375),
 ('라미', 0.42408578661221585),
 ('학보', 0.423844042948899),
 ('세뱃돈', 0.41938121923030325),
 ('분반', 0.41502840050637846)]

[('브레이브걸스', 2.22933911185193),
 ('단태', 2.0792541562070443),
 ('피셋', 1.703284982834478),
 ('신도시', 1.642462685846044),
 ('투기', 1.6165370239971752),
 ('검찰', 1.5690809043020542),
 ('결석', 1.5168880880759443),
 ('토지', 1.478933863061037),
 ('벚꽃', 1.4000063024463174),
 ('로나', 1.3672450068133981),
 ('마사', 1.3500116675752236),
 ('몰수', 1.2590418314014218),
 ('롤린', 1.2443783911042554),
 ('하윤', 1.2395557600221048),
 ('에이프릴', 1.2199456431016544),
 ('장관', 1.099773246730592),
 ('역주행', 1.0995421181074967),
 ('유신', 1.049941839395576),
 ('서진', 0.99970779535535),
 ('꼬북좌', 0.9836643072225276),
 ('수사', 0.9438734921055012),
 ('압수수색', 0.9182709420771997),
 ('자진', 0.8843710011701326),
 ('소급', 0.8766788875087869),
 ('버닝썬', 0.828980397868778),
 ('민주화', 0.8229285479641979),
 ('성매매', 0.811031207467534),
 ('룸살롱', 0.7873029929898611),
 ('향영', 0.7253117704861348),
 ('그래미', 0.7150509382790209),
 ('박수홍', 0.6941311631646634),
 ('임효준', 0.6317820125674505),
 ('수련', 0.614662265168781),
 ('쁘걸', 0.6074678333528786),
 ('동선', 

[('서예지', 1.9022807834692488),
 ('슈퍼리그', 1.6895519677147233),
 ('클로즈', 1.6880632934864688),
 ('김정현', 1.2730761500460281),
 ('비제', 1.1448339147424684),
 ('만우절', 1.1299672608995588),
 ('벼락치기', 0.8972771472427674),
 ('권혁', 0.4997246313270325),
 ('파상', 0.4670631125631868),
 ('분포', 0.4465461535817043),
 ('너비', 0.43658398039753943)]

[('참가자', 2.0620549547283336),
 ('퍼즐', 2.061422982385385),
 ('공혁준', 2.0193505845033326),
 ('이루리', 1.7752761958177912),
 ('정민', 1.758700161419743),
 ('퇴소', 1.7352013308542047),
 ('또즐', 1.7010357834601493),
 ('니갸르', 1.5548242004092578),
 ('파이', 1.5152887325168194),
 ('자진', 1.5046535945302328),
 ('사물함', 1.4891681972729147),
 ('학생회', 1.4786129546458162),
 ('전기', 1.2066400820413237),
 ('범인', 1.0835541224993812),
 ('코난', 1.0285868555594113),
 ('성당', 0.9012279189844271),
 ('에어컨', 0.8047932944798283),
 ('어린이', 0.7861394025069605),
 ('실종', 0.7526812093728193),
 ('캣맘', 0.7436837301002479),
 ('무죄', 0.6489785383501229),
 ('어버이날', 0.6076281219154295),
 ('약육강식', 0.5902651138496333),
 ('호날두', 0.574112523473142),
 ('퇴', 0.5604843885505018),
 ('육지', 0.5199889921983183),
 ('어린이날', 0.4606035918840763),
 ('최면', 0.4170355382815962)]

[('텀페', 1.606603203188053),
 ('관회', 1.3746085911616708),
 ('광학', 1.2967518608025441),
 ('독일', 1.242447823527372),
 ('팬덤', 1.201700999169101),
 ('소희', 1.1362068081919847),
 ('부정행위', 1.094260938677591),
 ('유로', 0.9619952825122491),
 ('메리트', 0.9043947269187297),
 ('중회', 0.6250872479194971),
 ('경통', 0.60320749747943),
 ('국무', 0.566041043992044),
 ('사프컴', 0.5319942501084838),
 ('현주', 0.5215723113458508),
 ('에고', 0.49777081290346725),
 ('잉글랜드', 0.49256289396082537),
 ('할미', 0.47724325105923293),
 ('반타작', 0.45792501046152534),
 ('쁘걸', 0.4462492446186387),
 ('떡밥', 0.4452337671503154),
 ('미분', 0.4445983569019486),
 ('곡선', 0.44408952348419106),
 ('리믹스', 0.4418088480118401),
 ('펩시', 0.4320645626434206),
 ('미적분', 0.4170023692209237),
 ('프제', 0.41329545502830006),
 ('열역학', 0.4113587420406447)]

### 단어별 검토

In [450]:
_word = '피셋'

- 월별 점수

In [451]:
Points[:,idx_extended[_word]]

array([ 0.        , -1.21918944, -1.37305272,  1.70328498, -1.34172567,
       -1.42083641, -1.34372986])

- 전체 등장 횟수

In [452]:
bow_extended[idx_extended[_word]]

746

- 월별 등장 횟수

In [453]:
BoW_mat[:,idx_extended[_word]]

array([ 79.,  50.,  36., 489.,  45.,  11.,  36.])

- 월별 게시글 수

In [454]:
posts_extended[:,idx_extended[_word]]

array([11., 14.,  8., 51., 10.,  4.,  8.])

- 지수 확인

In [447]:
end = 6
pp3(BoW_mat[end-1,idx_extended[_word]],BoW_mat[end,idx_extended[_word]],
    posts_extended[end-1,idx_extended[_word]], posts_extended[end,idx_extended[_word]],
    idx_extended[_word])

상승률지수 : 1.463536799078829
언급지수   : 0.04921103108035467
게시글상승 : 0.0565289497131827
게시글수   : -0.251312339887548
패널티     : -0.021212579182274108
최종값     : 1.2967518608025441


- 내용 검색 툴

In [101]:
def search_cd(word, x):
    try:
        return True if word in [y[0] for y in x] else False
    except:
        return False

def s_search(word, month, show_urls=True):
    df_month = df2[df2.upload_date.apply(lambda x: True if x.month==month else False)]
    title_cd = df2.title_tokenized.apply(lambda x: search_cd(word,x))
    text_cd = df2.text_tokenized.apply(lambda x: search_cd(word,x))
    comments_cd = df2.comments_tokenized.apply(lambda x: search_cd(word,x))
    comments_writer_cd = df2.comments_writer_tokenized.apply(lambda x: search_cd(word,x))
    df = df_month[title_cd | text_cd | comments_cd | comments_writer_cd]
    if show_urls==True:
        [print('http://www.ssodam.com/content/'+str(number)) for number in df.num]
    return df

def numofposts(word, month):
    df = s_search(word,month,show_urls=False)
    return len(df)

In [102]:
s_search('모닝콜',1)

NameError: name 'df2' is not defined

---

,num,board,title,text,writer,upload_date,upload_time,view,likes,dislikes,comments,comments_writer,comments_cnt,text_tokenized,title_tokenized,comments_tokenized,comments_writer_tokenized
0,1082901,익게2,해피뉴이어,2021년은 모두 행복한 한해가 되길,익명,2021-01-01,00:00,1343.0,195.0,0.0,"앗, 성지다, 첫글 ㅊㅊ, 추천 준다, 내 24살은 인생에서 가장 빛나는 날로 만들...",,20.0,"[(2021년, Number), (은, Foreign), (모두, Noun), (행...","[(해피뉴이어, Noun)]","[(앗, Noun), (,, Punctuation), (성지, Noun), (다, ...",[]
1,1082902,익게2,2021년 새해복 많이받으세요,모든 일이 잘 되기를 12시 땡,익명,2021-01-01,00:00,107.0,3.0,0.0,늦었네,"2빠다 ㅎㅎ, 내년에 도전한다",3.0,"[(모든, Noun), (일이, Modifier), (잘, Verb), (되, Ve...","[(2021년, Number), (새해, Noun), (복, Noun), (많이, ...","[(늦었네, Verb)]","[(2, Number), (빠다, Noun), (ㅎㅎ, KoreanParticle)..."
2,1082903,익게2,첫글은 내꼬,예비회계사 나다미,익명,2021-01-01,00:00,139.0,1.0,0.0,"ㄹㅇ 1등이네 ㅋㅋ, 실패",ㅜㅜ,3.0,"[(예비, Noun), (회계사, Noun), (나, Noun), (다미, Noun)]","[(첫, Modifier), (글, Noun), (은, Josa), (내꼬, Noun)]","[(ㄹㅇ, KoreanParticle), (1등, Number), (이네, Fore...","[(ㅜㅜ, KoreanParticle)]"
3,1082905,익게2,땡ㅎㅎㅎㅎㅎㅎㅎ,1등,익명,2021-01-01,00:00,39.0,0.0,0.0,,,0.0,"[(1등, Number)]","[(땡, Noun), (ㅎㅎㅎㅎㅎㅎㅎ, KoreanParticle)]",[],[]
4,1082906,익게2,어디 카운트 다운 하는 곳 없냐,언제바껴,익명,2021-01-01,00:00,84.0,0.0,0.0,232번 불교방송,,1.0,"[(언, Modifier), (제바, Noun), (껴, Verb)]","[(어디, Noun), (카운트, Noun), (다운, Noun), (하는, Ver...","[(232, Number), (번, Noun), (불교, Noun), (방송, No...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77975,1200285,익게2,아버지께서 칼럼 쓰셨는데 댓글 한 번씩만 달아줄 수 있을까,2222408544049우리 아버지께서 이번에 짧은 칼럼처럼 블로그에 쓰시는 거 맡...,익명,2021-06-30,23:43,5802.0,374.0,11.0,"비댓으로 달래 너무 귀엽다, 이 글 조회수 늘리려면 제목 바꾸는거 추천 광고글인줄 ...","헉 피드백 반영했어 너무 고마워, 엇 아마 확인은 못하실 텐데 비댓이든 공개댓이든 ...",75.0,"[(2222408544049, Number), (우리, Noun), (아버지, No...","[(아버지, Noun), (께서, Josa), (칼럼, Noun), (쓰셨는데, V...","[(비댓, Noun), (으로, Josa), (달래, Noun), (너무, Adve...","[(헉, Adverb), (피드백, Noun), (반영, Noun), (했어, Ve..."
77976,1200287,익게2,프로포폴 하면 기분이 좋음,하는거 신기하네하면 힙해보여서 하는건가,익명,2021-06-30,23:47,373.0,1.0,1.0,썰로 들은 거라 정확하지는 않는데 지방흡입할 때 고통 줄이려고 프로포폴 투약하다가 ...,,5.0,"[(하는거, Verb), (신기하네하면, Adjective), (힙, Noun), ...","[(프로포폴, Noun), (하면, Verb), (기분, Noun), (이, Jos...","[(썰로, Verb), (들은, Verb), (거, Noun), (라, Josa),...",[]
77977,1200288,익게2,취업 후기 쓰려는데 안올라간다ㅠ,손다쳐서 폰으로 열심히 적었는데나한테왜 이래ㅠ혹시 뭔가 지켜야 하는 양식이 있나요ㅠ...,익명,2021-06-30,23:49,387.0,3.0,0.0,"중간에 임티 넣었었어, 임티, 1 이모티콘 빼기2 사진 많으면 업로드 될 동안 조금...",이건 되네,7.0,"[(손, Noun), (다쳐서, Verb), (폰, Noun), (으로, Josa)...","[(취업, Noun), (후기, Noun), (쓰려는데, Verb), (안, Ver...","[(중간, Noun), (에, Josa), (임티, Noun), (넣었었어, Ver...","[(이건, Noun), (되네, Verb)]"
77978,1200291,익게2,간떨어지는동거 보는 사람,아 이번주 기대했는데 예고편...,익명,2021-06-30,23:55,138.0,6.0,0.0,"너무 오글거리는데 장기용때매 봄 ㅎㅎ, 난 계선우땜에 봐ㅠㅠ, 나도 계선우 때문에 ...",,3.0,"[(아, Exclamation), (이번, Noun), (주, Noun), (기대했...","[(간, Noun), (떨어지는, Verb), (동거, Noun), (보는, Ver...","[(너무, Adverb), (오글거리는데, Verb), (장기, Noun), (용때...",[]


## ETC

In [962]:
Posts_mat = np.zeros((7,len(idx_extended)))
for i, key in enumerate(idx_extended):
    print(i)
    for j in range(1,7):
        print(j)
        Posts_mat[j,i] = numofposts(key,i)

0
1
2
3
4
5
6
1
1
2
3
4
5
6
2
1
2
3
4
5
6
3
1
2
3
4
5
6
4
1


KeyboardInterrupt: 

In [ ]:
with open('posts_mat.pkl','wb') as f:
    pickle.dump(Posts_mat,f)